In [10]:
"""
Import images
"""

import glob
import numpy as np
import pandas as pd
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from sklearn.preprocessing import LabelEncoder


def imp_img(img_name):
    # read
    img = imread(img_name)
    # convert to gray
    if len(img.shape) == 3:
        img = rgb2gray(img)
    return img

img_size = 30

path = "data"
train_names = sorted(glob.glob(path + "/trainResized/*"))
train_labels_index = pd.DataFrame.from_csv('trainLabels.csv')
train_files = np.ones((len(train_names), img_size, img_size))
train_labels = np.ones((len(train_names),)).astype(str)
for i, name_file in enumerate(train_names):
    
    image = imp_img(name_file)
    train_files[i, :, :] = image
    train_labels[i] = train_labels_index.loc[int(name_file.split('.')[0].split('/')[-1])]['Class']

test_names = sorted(glob.glob(path + "/testResized/*"))
test_files = np.ones((len(test_names), img_size, img_size))
for i, name_file in enumerate(test_names):
    image = imp_img(name_file)
    test_files[i, :, :] = image

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
print(train_files.shape, test_files.shape)
print(np.unique(train_labels))

(6283, 30, 30) (6220, 30, 30)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61]


In [11]:
"""
Configure train/test
"""

cv_prob = np.random.sample(train_files.shape[0])
train_cv_ind = cv_prob < 0.75
test_cv_ind = cv_prob >= 0.75
X_train, y_train = train_files[train_cv_ind, :, :], train_labels[train_cv_ind]
X_test, y_test = train_files[test_cv_ind, :, :], train_labels[test_cv_ind]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4714, 30, 30) (4714,)
(1569, 30, 30) (1569,)


In [12]:
"""
Compile Model
"""

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

np.random.seed(1337)  # for reproducibility

batch_size = 128
nb_classes = 62
nb_epoch = 12

# input image dimensions
img_rows, img_cols = img_size, img_size
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 3
# convolution kernel size
nb_conv = 6

# the data, shuffled and split between train and test sets
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta')

X_train shape: (4714, 1, 30, 30)
4714 train samples
1569 test samples


In [5]:
"""
CV model
"""
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [7]:
"""
Get accuracy
"""
predicted_results = model.predict_classes(X_test, batch_size=batch_size, verbose=1)
print(predicted_results)